# TastyTradeAPI – Example of Use in Jupyter Notebook

This document shows how to use the **Tastytrade API** through the `TastyTradeAPI` library included in the project.  

⚠️ **Important Note**:  
In this demo session I do not have a **linked trading account**, therefore **it is not possible to place real orders** or query certain account metrics.  
Nevertheless, all functions are implemented and accompanied by **simulated output examples**, so you can fully understand how to use them without any risk of error.  

The main API documentation is included in the source code itself (`api.py`), with explanations and examples for each function block.  

---

# Table of Contents

- [TastyTradeAPI – Example of Use in Jupyter Notebook](#tastytradeapi--example-of-use-in-jupyter-notebook)
- [Project Structure](#project-structure)
- [Import the API](#import-the-api)
- [Login as Client](#login-as-client)
- [Check if the Session is Still Active](#check-if-the-session-is-still-active)
- [Account Information](#account-information)
  - [Basic Client Information](#basic-client-information)
  - [Account Metrics](#account-metrics)
  - [Positions Data](#positions-data)
  - [Transactions Data](#transactions-data)
- [Execute Orders](#execute-orders)
- [Stream Historical Market Data](#stream-historical-market-data)
  - [Parameters](#parameters)
  - [Example](#example)
- [Stream Real-Time Data](#stream-real-time-data)
  - [Parameters](#parameters-1)
  - [Response](#response)


---

## Project Structure

- **TastyTradeAPI/**  
  - `__init__.py` → module initializer.  
  - `api.py` → API implementation with all documented functions.  

<br>

- **example_es.ipynb** → example notebook in Spanish.  
- **example_en.ipynb** → example notebook (this document) in English.  
- **requirements.txt** → external libraries required to run the project.  
- **.env** → private credentials and configuration (Tastytrade username and password).  
- **venv/** → Python virtual environment.  


## Import the API

To work with this API you must have the **TastyTradeAPI** repository in the same directory as your project and import it as shown below.  

In the example, the alias `ts` is assigned, although you can use any valid Python name.  

The credentials (`username` and `password`) correspond to the email and password of your Tastytrade account.  
For security reasons, it is recommended to load them as **environment variables** instead of writing them directly into the code. However, they can also be passed as *strings* if you prefer.


In [2]:
from TastyTradeAPI.api import TastyTradeAPI as ts
import os
from dotenv import load_dotenv

load_dotenv()

USER = os.environ.get("MAIL")
PASS = os.environ.get("PASS")

## Login as Client

To initialize the API client, you need to provide the **username** and **password** of your Tastytrade account.  

It is not mandatory to have an active trading account: logging in is enough.  
Keep in mind that without an active account you will not be able to execute trades or query key metrics, although you can still access **historical market data**.  

The generated client is a dictionary containing the session information, for example (`the token is not real`):


In [11]:
api = ts()
client = api.Client(USER,PASS)

print(client)

{"session_token": "fz9a8b7c6d5e4f3g2h1i0jklmnopqrstuVWXY", "account_number": "No Account"}


## Revisar si la sesión sigue activa

La autenticación está basada en **tokens simples** (no en OAuth2), por lo que la sesión puede caducar tras unos minutos de inactividad.  

La función correspondiente devuelve `True` si el cliente sigue activo y `False` en caso contrario.  
En caso de que la sesión haya expirado, será necesario volver a **inicializar un nuevo cliente** con las credenciales. 

In [4]:
state = api.still_connected(client)

print(state)

True


## Account Information

Once the session is started, you can query the client’s basic data and the main account metrics.  

### Basic Client Information
Returns general data such as **name**, **email**, **country of origin**, and the **net worth** associated with the profile.


In [5]:
info = api.client_info(client)

print(info)

{'name': 'Pablo Miñana', 'email': 'gandpablo04@gmail.com', 'citizenship_country': 'ESP', 'liquid_net_worth': 0}


### Account Metrics

The metrics provide a quick overview of the financial status of the client’s account:

- **Balance** → total available funds in the account.  
- **Equity Buying Power** → available buying power for trading stocks.  
- **Derivative Buying Power** → available buying power for trading derivatives (options, futures).  
- **Liquidity** → current liquidity of the account, considering assets and liabilities.  
- **Total Fees** → sum of all commissions generated so far.  


In [ ]:
balance = api.balance(client)

equityBP = api.equity_BP(client)

derivativeBP = api.derivative_BP(client)

liquidity = api.liquidity(client)

total_fees = api.total_fees(client)

print('Balance:', balance)
print('Equity BP:', equityBP)
print('Derivative BP:', derivativeBP)
print('Liquidity:', liquidity)
print('Total Fees:', total_fees)

```Example Output```

```python
Balance: 12534.25
Equity BP: 10000.00
Derivative BP: 2500.00
Liquidity: 8750.50
Total Fees: 35.75
```
---

### Positions Data

These functions allow you to query detailed information about the open positions in the client’s Tastytrade account.  

Each position includes the following fields:

- **symbol** → Stock ticker.  

- **quantity** → Number of shares in the position.  

- **type** → Position type: `Long` or `Short`.  

- **profit** → Profit (>0) or loss (<0), calculated based on the difference between the current price (*mark*) and the entry price (*cost basis*), adjusted by the quantity and the multiplier.  

  **Formulas:**  

    - **Long Positions**  
  ```
   Profit = ( Mark − Cost Basis ) × Quantity × Multiplier
  ```
    - **Short Positions**  
  ```
   Profit = ( Cost Basis − Mark ) × Quantity × Multiplier
  ```


- **avg_entry_price** → Average entry price.  

- **current_price** → Current stock price.  

- **opened_at** → Date and time when the position was opened.  

- **updated_at** → Last date and time when the position was updated.  

The function returns a **dictionary**, where the key is the *ticker* and the value is another dictionary containing all the information for the corresponding position.


In [ ]:
positions = api.all_positions(client)

print(positions)

```Example Output```

```python
{
  "AAPL": {
    "symbol": "AAPL",
    "quantity": 100,
    "type": "Long",
    "profit": 1250.00,
    "avg_entry_price": 150.00,
    "current_price": 162.50,
    "opened_at": "2025-09-10 14:30:00",
    "updated_at": "2025-09-17 09:15:00"
  },
  "TSLA": {
    "symbol": "TSLA",
    "quantity": 50,
    "type": "Short",
    "profit": -750.00,
    "avg_entry_price": 240.00,
    "current_price": 255.00,
    "opened_at": "2025-09-12 10:05:00",
    "updated_at": "2025-09-17 09:15:00"
  }
}


It is also possible to query a specific position by its *ticker*.  

The function returns a **dictionary with the position information** if it exists, or `None` otherwise.


In [ ]:
pos = api.check_position(client, "AAPL")

print(pos)

```Example Output```

```python
{
  "symbol": "AAPL",
  "quantity": 100,
  "type": "Long",
  "profit": 1250.00,
  "avg_entry_price": 150.00,
  "current_price": 162.50,
  "opened_at": "2025-09-10 14:30:00",
  "updated_at": "2025-09-17 09:15:00"
}
```
---

### Transactions Data

These functions allow you to review the transactions made in the client’s Tastytrade account.  

Each transaction includes the following fields:

- **id** → Unique identifier of the transaction.  

- **transaction_type** → Type of transaction (e.g. `Buy`, `Sell`, `Dividend`, `Fee`...).  

- **description** → Description of the operation (e.g. *Buy to Open*).  

- **quantity** → Number of shares or contracts involved.  

- **price** → Execution unit price.  

- **value** → Total value of the transaction.  

- **date** → Date and time when the operation was executed.  

The function returns a **dictionary** where the key is the *ticker* and the value is a **list of transactions** associated with that symbol.


In [ ]:
transactions = api.all_transactions(client)

print(transactions)

`Example Output`

```python
{
  "AAPL": [
    {
      "id": "TX001",
      "transaction_type": "Buy",
      "description": "Buy to Open",
      "quantity": 100,
      "price": 150.00,
      "value": 15000.00,
      "date": "2025-09-10 14:30:00"
    }
  ],
  "TSLA": [
    {
      "id": "TX002",
      "transaction_type": "Sell",
      "description": "Sell to Close",
      "quantity": 50,
      "price": 240.00,
      "value": 12000.00,
      "date": "2025-09-12 10:05:00"
    }
  ]
}


It is also possible to query the transactions associated with a specific *ticker*.  

The function returns a **list of transactions** if they exist, or `None` otherwise.


In [ ]:
txs = api.check_transaction(client, "AAPL")

print(txs)

`Example Output`

```python
[
  {
    "id": "TX001",
    "transaction_type": "Buy",
    "description": "Buy to Open",
    "quantity": 100,
    "price": 150.00,
    "value": 15000.00,
    "date": "2025-09-10 14:30:00"
  }
]

```
---


## Execute Orders

To place an order you must specify:

- **symbol (ticker)**: The asset to trade, for example `"AAPL"`.  

- **quantity (val)**: The number of shares or contracts.  

- **order-type**: Whether the order is `'Long'` (long position) or `'Short'` (short position).  

- **action**: Whether you want to open a position (`'Start'`) or close it (`'End'`).  

- **time-in-force**: Determines how long the order remains active:  
  - `'Day'` → valid only until market close.  
  - `'GTC'` → remains active until executed or canceled.  
  - `'GTD'` → valid until a specific date (requires additional parameter `gtc-date`).  
  - `'Ext'` → valid only during extended hours (equities).  
<br>

- **otype**: Defines how the order is executed:  
  - `'Market'` → executes at the best available price, without specifying a price.  
  - `'Limit'` → requires a specified price and executes if the market reaches it.  
  - `'Stop'` → triggers as a market order when a stop price is reached.  
  - `'Stop Limit'` → triggers as a limit order when a stop price is reached.  
<br>

For example: Buy 10 shares of Apple (AAPL) with a market order (`Market`), valid until today’s close (`Day`).  


In [ ]:
order_result = api.order(
    Client=client,
    ticker="AAPL",        
    val=10,               
    order_type="Long",    
    action="Start",       
    time_force="Day",     
    otype="Market"        
)

print(order_result)

`Example Output`

```python
{
  "Status": "Filled",
  "Size": 10,
  "Fees": 1.25,
  "Commission": 0.50,
  "NewBuyingPower": 9850.00,
  "ReceivedAt": "2025-09-17 14:35:00"
}
```

When an order is executed, a dictionary is returned with the following fields:

- **Status** → Order status (`Routed`, `Filled`, `Cancelled`, etc.).  

- **Size** → Quantity sent in the order.  

- **Fees** → Total fees associated with the trade.  

- **Commission** → Broker’s commission.  

- **NewBuyingPower** → Buying power available after the order.  

- **ReceivedAt** → Date and time when the order was received.  


## Stream Historical Market Data

This function connects to the **DXFeed** *market data* provider (integrated with Tastytrade).  

⚠️ **Important**: <br>  
**It may take several seconds** depending on the number of tickers requested and the amount of data.  
It is not unusual for it to take `more than 20 seconds` for 50 data points of a single ticker.  
If you request many data points for multiple tickers, it may even run for `more than 1 minute`.  

The workflow is as follows:

1. With the **client**, a DX *token* is obtained.  

2. Using that token, the historical candles for the requested **tickers** are requested via **websocket**.  

3. The received data is formatted into a `DataFrame` with the selected variables.  

4. Finally, the results are returned in a dictionary where each key is a ticker and the value is its `DataFrame`.  

⚠️ **Important DXFeed Limitation**:  
By design, it only allows requesting data from the **current moment backwards** up to the maximum number requested (`max_data`).  
It is not possible to request arbitrary intervals (e.g., from one date to another).  
That is why there is a helper function `values_from_data`, which calculates how many records are needed to reach the desired date.  

---

##### Parameters

- **client** → Tastytrade client defined at the beginning.  

- **tickers** → List of asset symbols you want to query.  
  Example: `["AAPL", "MSFT"]`.  

- **interval** → Defines the granularity of the data. It is passed as number + unit:  
  - `s` → seconds  
  - `m` → minutes  
  - `h` → hours  
  - `d` → days  
  - `w` → weeks  
  - `mo` → months  
  Example: `"5m"` = 5-minute candles.  
<br>

- **vars** → List of variables you want to retrieve. Available values (specify the ones you need):  
  - `'open'` → Opening price (`Open`)  
  - `'high'` → Highest price (`High`)  
  - `'low'` → Lowest price (`Low`)  
  - `'close'` → Closing price (`Close`)  
  - `'volume'` → Volume (`Volume`)  
<br>

- **max_data** → Maximum number of records to request.  
  It is usually adjusted with the `values_from_data` function to obtain exactly the amount of data needed up to the desired date.  

##### `Example`:  
`Download 50 candles every 5 minutes for AAPL and TSLA, only open, close, and volume.`  

The function returns a **dictionary** with the following structure:  

```python
{
    "AAPL": DataFrame with columns [Open, Close, Volume, Date],
    "TSLA": DataFrame with columns [Open, Close, Volume, Date],
}


In [6]:
tickers = ['AAPL','TSLA']
interval = '5m'
max_data= 50
vars = ['open', 'close','volume']

historical = api.get_historical(client, tickers, interval, vars,max_data)

In [7]:
historical['TSLA'].head(6)

,Open,Close,Volume,Date
31,421.39,421.4199,5301.0,2025-09-16 21:30:00
32,421.3875,421.3,4699.0,2025-09-16 21:25:00
33,421.201,421.345,5873.0,2025-09-16 21:20:00
34,421.6378,421.2003,5399.0,2025-09-16 21:15:00
35,421.359,421.6,5233.0,2025-09-16 21:10:00
36,421.39,421.31,6369.0,2025-09-16 21:05:00


You can calculate `max_data` based on a date (`values_from_data`).  

This function automatically calculates how many points (`max_data`) need to be requested in `get_historical` to cover from a **start date** up to **now**, depending on the interval.  

Required parameters:  
- **interval** → candle size (`'5m'`, `'1h'`, `'1d'`, `'1w'`, `'1mo'`).  
- **date** → start date (`datetime`).  

It returns an integer that is then passed as `max_data` in `get_historical` to retrieve all the data from that date until the present.  

`Example:` Get daily candles (`'1d'`) from **April 21, 2025** up to today:  


In [8]:
from datetime import datetime, timezone

date = datetime(2025, 4, 21, 12, 0, tzinfo=timezone.utc)
needed = api.values_from_data("1d", date)

print('Como max_data usaremos: ',needed)

Como max_data usaremos:  150


## Stream Real-Time Data

`RealTimeStreamer` is a simple wrapper to connect to the **DXFeed websocket** (via Tastytrade) and receive **real-time quotes** without the user having to manually manage the connection.  
It is designed for dashboards, bots, or scripts that need real-time prices for multiple symbols.  

**Returns** a dictionary with the **`askPrice`** of each requested quote.  
**Connection:** it is **open and persistent**; therefore, sometimes **fuzzy or incomplete messages** may arrive, some connections may **take time to open or close**, and **not all symbols** update at the same time (some messages **may arrive with a delay** compared to others).  

**How it works:** the object continuously sends messages until the user stops it; it is usually used with **infinite `while` loops**, or with **`while` + `break` / `for`** if you only want it active for a set period of time.  

```The data is contained in the streamer variable called data --> object.data```

---

#### Parameters

- `client`: tastytrade client defined at the beginning.  
- `tickers`: list of symbols, e.g. `["AAPL", "TSLA"]`.  
- `verbose` *(bool, optional; default `False`)*: if set to `True`, **prints all websocket messages** and the connection status (useful for debugging).  

---

#### Response

- **`data`** *(dict)*: latest **`askPrice`** per symbol, e.g.  
  ```python
  {"AAPL": 238.00, "MSFT": 510.05}
  ```

  >
  > **``` This is the variable you will work with```**  
  >


In [9]:
import time

tickers = ['AAPL','TSLA','MSFT','AMZN','GOOGL']   

stream = api.RealTimeStreamer(client,tickers)

stream.start()

for _ in range(10):
    data = stream.data
    print(data)
    time.sleep(1)
    
stream.stop()

{'AAPL': 238.1, 'TSLA': 422.61, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.61, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.6, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.6, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.6, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.6, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.58, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.56, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.56, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}
{'AAPL': 238.1, 'TSLA': 422.56, 'MSFT': 510.19, 'AMZN': 233.85, 'GOOGL': 251.43}


In [12]:
tickers = ['AAPL']

stream = api.RealTimeStreamer(client,tickers,verbose=True)

stream.start()


while True:
    data = stream.data
    print(data)

    if data and data['AAPL'] > 238:
        print("The condition happened, stopping the stream.")
        stream.stop()
        break

    time.sleep(1)

-->Connection opened
Message received --> {"type":"SETUP","channel":0,"keepaliveTimeout":120,"acceptKeepaliveTimeout":120,"version":"1.0-1.2.3-20240912-171526"}
Message received --> {"type":"AUTH_STATE","channel":0,"state":"UNAUTHORIZED"}
Message received --> {"type":"AUTH_STATE","channel":0,"state":"AUTHORIZED","userId":"abcd1234-5678-90ab-cdef-112233445566"}
Message received --> {"type":"CHANNEL_OPENED","channel":3,"service":"FEED","parameters":{"contract":"AUTO","subFormat":"LIST"}}
Message received --> {"type":"FEED_CONFIG","channel":3,"dataFormat":"COMPACT","aggregationPeriod":0.1}
Message received --> {"type":"FEED_CONFIG","channel":3,"dataFormat":"COMPACT","aggregationPeriod":0.1,"eventFields":{"Quote":["askPrice","eventSymbol"]}}
Message received --> {"type":"FEED_DATA","channel":3,"data":["Quote",[241.15,"AAPL"]]}
{'AAPL': 241.15}
The condition happened, stopping the stream.
-->Connection closed
-->Streamer stopped


`The userId variable is randomly generated so it is not visible`